In [0]:
"""
gold_summary.py

Aggregates cleaned Silver layer data to generate a vendor-level summary for reporting.

- Counts total inventory items and shipments per vendor
- Tracks last shipment date
- Outputs to: /mnt/delta/gold/vendor_summary_clean

Table registered as: vendor_summary_clean
"""

In [0]:
%run "/Repos/brucejenks@live.com/databricks-pipelines/pipeline1_batch_delta/utils/utils_write_delta"


In [0]:
# Gold: Final Vendor Summary Writer

from pyspark.sql import SparkSession
from pyspark.sql.functions import col, max, countDistinct, current_timestamp

spark = SparkSession.builder.getOrCreate()

# Load Silver sources
df_finance = spark.read.format("delta").load("/mnt/delta/silver/final_vendor_summary_prep")
df_registry = spark.read.format("delta").load("/mnt/delta/silver/vendor_registry_clean").alias("registry")
df_compliance = spark.read.format("delta").load("/mnt/delta/silver/vendor_compliance_clean").alias("compliance")

# Aggregate and join
df_gold = (
    df_finance
    .join(df_registry, on="vendor_id", how="left")
    .join(df_compliance, on="vendor_id", how="left")
    .groupBy("vendor_id", "vendor_name")
    .agg(
        countDistinct("invoice_id").alias("total_invoices"),
        max("due_date").alias("latest_due_date"),
        max("invoice_date").alias("latest_invoice_date"),
        max(col("compliance.last_audit_date")).alias("last_audit_date"),
        max(col("compliance.compliance_score")).alias("compliance_score"),
        max(col("compliance.status")).alias("compliance_status"),
        max(col("registry.industry")).alias("industry"),
        max(col("registry.headquarters")).alias("headquarters"),
        max(col("registry.onwatchlist")).alias("onwatchlist"),
        max(col("registry.registrationdate")).alias("registration_date"),
        max(col("registry.tier")).alias("tier")
    )
)

# Add run timestamp
df_gold = df_gold.withColumn("pipeline_run_timestamp", current_timestamp())

# Set output path
target_path = "/mnt/delta/gold/final_vendor_summary"

# Save and register Gold table, partitioned by 'tier'
write_df_to_delta(
    df=df_gold,
    path=target_path,
    mode="overwrite",
    register_table=True,
    merge_schema=True,
    partition_by=["tier"],
    verbose=True
)

# ---- Summary Statistics (Print) ----
print("📊 Gold Table Summary Stats:")
df_gold.selectExpr(
    "count(*) as total_records",
    "count(distinct vendor_id) as distinct_vendors",
    "min(registration_date) as earliest_registration",
    "max(registration_date) as latest_registration"
).show(truncate=False)

# ---- Track Run History ----
df_log = df_gold.select(
    current_timestamp().alias("run_time"),
    countDistinct("vendor_id").alias("vendor_count"),
    countDistinct("vendor_name").alias("vendor_name_count")
)

log_path = "/mnt/delta/logs/final_vendor_summary_runs"

write_df_to_delta(
    df=df_log,
    path=log_path,
    mode="append",
    register_table=True,
    merge_schema=True,
    partition_by=None,
    verbose=True
)


In [0]:
display(df_log)